In [ ]:
import requests
import json
import re
from bs4 import BeautifulSoup as bs4
from IPython.display import display, HTML
import os
import pandas as pd

In [ ]:
client_id = os.environ.get("DEVIANTART_CLIENT_ID")
client_secret = os.environ.get("DEVIANTART_CLIENT_SECRET")

In [ ]:
token_request = requests.post("https://www.deviantart.com/oauth2/token", params= {
    "client_id": client_id,
    "client_secret": client_secret,
    "grant_type": "client_credentials"
} )

In [ ]:
token = token_request.json()["access_token"]

In [ ]:
offset = 0
limit = 24
esk_request = requests.get("https://www.deviantart.com/api/v1/oauth2/gallery/all", 
headers={
    'Authorization': 'Bearer {}'.format(token)
    }, 
    params= {
    "username": "esk-masterlist",
    "offset": offset,
    "limit": limit
}).json()

In [ ]:
esk_requests = [esk_request["results"]]
while esk_request["has_more"]:
    offset += limit
    esk_request = requests.get("https://www.deviantart.com/api/v1/oauth2/gallery/all", 
        headers={
            'Authorization': 'Bearer {}'.format(token)
            }, 
            params= {
            "username": "esk-masterlist",
            "offset": offset,
            "limit": limit
        }).json()
    esk_requests.append(esk_request["results"])

In [ ]:
deviation_ids = [esk["deviationid"] for esks in esk_requests for esk in esks]

In [ ]:
start_index = 0
end_index = 50
esk_metadata_request = requests.post("https://www.deviantart.com/api/v1/oauth2/deviation/metadata", 
    data={
        "deviationids[]": deviation_ids[start_index:end_index]
    },
    headers={
        'Authorization': 'Bearer {}'.format(token)
    }).json()["metadata"]


In [ ]:
esk_metadata = [esk_metadata_request]
while end_index < len(deviation_ids):
    start_index += 50
    end_index += 50
    if end_index > len(deviation_ids):
        end_index = len(deviation_ids)
    esk_metadata_request = requests.post("https://www.deviantart.com/api/v1/oauth2/deviation/metadata", 
    data={
        "deviationids[]": deviation_ids[start_index:end_index]
    },
    headers={
        'Authorization': 'Bearer {}'.format(token)
    }).json()["metadata"]
    esk_metadata.append(esk_metadata_request)


In [ ]:
print(esk_metadata[0])

In [ ]:

def esk_record_from_web_response(raw_esk_record):
    try:
        parsed_html = bs4(raw_esk_record["description"])
        bold_texts = parsed_html.body.select('div > b', class_="legacy-journal")
        for bold in bold_texts:
            tmp = ""
            if bold != None:
                if bold.string != None:
                    if re.fullmatch(r"\s+", bold.string) != None:
                        continue
                    if bold.string == "designer":
                        continue
                    if bold.string == "ownership":
                        break
                    if bold.string == "past designs":
                        continue
                    if bold.next_sibling != None:
                        tmp = bold.next_sibling
                        if "username-with-symbol" in tmp:
                            continue
                        if str.endswith(tmp, ".") or str.endswith(tmp, "("):
                            tmp = tmp[:-1].strip()
                        #if "mutation" in bold.string:    
                        tmp = re.sub(r"\(.*\)", "", tmp)
                        raw_esk_record[bold.string] = tmp
        raw_esk_record.pop("description", None)
        owner_history = parsed_html.body.find('b', string= "owner history", recursive=True)
        raw_esk_record["owner count"] = len(owner_history.find_next_sibling("sub").findChildren("br")) + 1
        return raw_esk_record
    except:
        return None


In [ ]:
esk_descriptions = [{"title": esk["title"], "description": esk["description"]} for esks in esk_metadata for esk in esks if (re.fullmatch( r"\d{3,4}", esk["title"]) != None)]

In [ ]:
esk_list = []

for esk in esk_descriptions:
    cleaned = esk_record_from_web_response(esk)
    if cleaned != None:
        esk_list.append(cleaned)

In [ ]:
df = pd.DataFrame(esk_list)
print(df)